In [2]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
import os
import pandas as pd
import imageio
from scipy.optimize import minimize
from joblib import Parallel, delayed
import pickle

In [3]:
#Initialize parameters
N = 50 #Number of nodes
l = 3  # Number of topics
lambda_val = 0.5 #Decay parameter 

# Rebuild A_initial from graph metadata saved
A_initial = np.array([G.nodes[n]['initial_opinion'] for n in G.nodes])

# Load static arbitrary graph
with open("fixed_graph_arbitrary.pkl", "rb") as f:
    G = pickle.load(f)

print(f"Loaded fixed arbitrary graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges and {np.mean([d for _, d in G.degree()])} average degree.")

Loaded fixed arbitrary graph with 50 nodes and 532 edges and 21.28 average degree.


In [4]:
#Player Class and Simulation
class Player:
    def __init__(self, theta_low, theta_high, delta, beta, topic_index, agent_id, push_toward, l, kappa, gamma):
        self.theta_low = theta_low
        self.theta_high = theta_high
        self.delta = delta
        self.beta = beta
        self.topic_index = topic_index
        self.agent_id = agent_id
        self.push_toward = push_toward
        self.l = l
        self.kappa = kappa
        self.gamma = gamma
   
    #Link weight calculation
    def compute_w_link(self, A_updated):
        N = A_updated.shape[0]
        w_link_matrix = np.zeros((N, N))
        for i in range(N):
            for j in range(i + 1, N):
                distance = np.linalg.norm(A_updated[i] - A_updated[j])
                w_link_matrix[i, j] = np.exp(-distance)
                w_link_matrix[j, i] = w_link_matrix[i, j]
        return w_link_matrix
    
    #Simulating for intial node
    def send_message_single_agent(self, network, initial_node, message):
        active_nodes = set()
        claimed_nodes = {}
        A_updated = np.copy(A_initial)

        alignment = self.beta * (1 - np.linalg.norm(A_initial[initial_node] - message) / (2 * np.sqrt(self.l)))

        if alignment > self.theta_low:
            w_link_matrix = self.compute_w_link(A_updated)
            neighbors = list(network.neighbors(initial_node))
            Z_i = np.sum(w_link_matrix[initial_node, neighbors])
            neighbor_influence = np.sum([A_updated[j] * w_link_matrix[initial_node, j] for j in neighbors]) / Z_i if Z_i > 0 else 0  
            A_updated[initial_node] = (1 - self.kappa - self.gamma) * A_updated[initial_node] + self.kappa * message + self.gamma * neighbor_influence
            active_nodes.add(initial_node)
            claimed_nodes[initial_node] = self.agent_id

        timestep_count = 0
        while True:
            new_active_nodes = set()
            w_link_matrix = self.compute_w_link(A_updated)
            for node in active_nodes:
                for neighbor in network.neighbors(node):
                    if neighbor in claimed_nodes:
                        continue
                    if w_link_matrix[node, neighbor] > self.delta:
                        alignment = self.beta * (1 - np.linalg.norm(A_updated[neighbor] - message) / (2 * np.sqrt(self.l)))
                        if alignment > self.theta_low:
                            neighbors = list(network.neighbors(node))
                            Z_i = np.sum(w_link_matrix[node, neighbors])
                            neighbor_influence = np.sum([A_updated[j] * w_link_matrix[node, j] for j in neighbors]) / Z_i if Z_i > 0 else 0
                            A_updated[neighbor] = (1 - self.kappa - self.gamma) * A_updated[neighbor] + self.kappa * message + self.gamma * neighbor_influence
                            new_active_nodes.add(neighbor)
                            claimed_nodes[neighbor] = self.agent_id
            if not new_active_nodes:
                break
            active_nodes.update(new_active_nodes)
            timestep_count += 1
        #Adjusting influence for negative direction
        total_influence = self.push_toward*np.sum(A_updated[:, self.topic_index] - A_initial[:, self.topic_index])
        return total_influence, A_updated, claimed_nodes, timestep_count


    #Optimizing using base vectors over all nodes
    def optimize_strategy_single_agent(self, network, n_jobs=-1):
        #Base vectors
        base_vectors = [np.eye(self.l)[i] for i in range(self.l)] + [-np.eye(self.l)[i] for i in range(self.l)]
        sampled_messages = base_vectors + [np.random.uniform(-1, 1, self.l) for _ in range(3)]

        def evaluate_strategy(initial_node, message):
            influence, _, _, _ = self.send_message_single_agent(network, initial_node, message)
            return (initial_node, message, influence)

        all_jobs = [(node, message) for node in network.nodes for message in sampled_messages]

        results = Parallel(n_jobs=n_jobs)(
            delayed(evaluate_strategy)(node, msg) for node, msg in all_jobs
        )

        #Find optimal choice
        best_node, best_message, max_influence = max(results, key=lambda x: x[2])

        return best_node, best_message, max_influence

#Simulating diffusion t>=2
def simulate_diffusion(player, network, initial_node, message):
    active_nodes_history = []
    A_updated = np.copy(A_initial)
    active_nodes = set([initial_node])
    claimed_nodes = {initial_node: player.agent_id}
    active_nodes_history.append(set(active_nodes))

    influence_history = []
    #Simulate until influence can no longer spread
    while True:
        new_active_nodes = set()
        w_link_matrix = player.compute_w_link(A_updated)
        for node in active_nodes:
            for neighbor in network.neighbors(node):
                if neighbor in claimed_nodes:
                    continue
                if w_link_matrix[node, neighbor] > player.delta:
                    alignment = player.beta * (1 - np.linalg.norm(A_updated[neighbor] - message) / (2 * np.sqrt(player.l)))
                    if alignment > player.theta_low:
                        neighbors = list(network.neighbors(node))
                        Z_i = np.sum(w_link_matrix[node, neighbors])
                        neighbor_influence = np.sum([A_updated[j] * w_link_matrix[node, j] for j in neighbors]) / Z_i if Z_i > 0 else 0
                        A_updated[neighbor] = (1 - player.kappa - player.gamma) * A_updated[neighbor] + player.kappa * message + player.gamma * neighbor_influence
                        new_active_nodes.add(neighbor)
                        claimed_nodes[neighbor] = player.agent_id
        if not new_active_nodes:
            break
        active_nodes.update(new_active_nodes)
        active_nodes_history.append(set(active_nodes))
        #Adjusting influence for negative direction
        influence_t = player.push_toward*np.sum(A_updated[:, player.topic_index] - A_initial[:, player.topic_index])
        influence_history.append(influence_t)

    return active_nodes_history, A_updated, claimed_nodes, influence_history

#Visualizing network diffusion
def visualize_single_agent_diffusion(G, active_nodes_history, A_updated, claimed_nodes, topic_index, message):
    topic_names = {0: "Affirmative Action", 1: "Gun Permits", 2: "Political Party ID"}
    topic_name = topic_names.get(topic_index, "Unknown Topic")
    max_timesteps = len(active_nodes_history)
    pos = nx.spring_layout(G, seed=42)

    for t in range(max_timesteps):
        plt.figure(figsize=(8, 6))
        node_colors = []
        current_claimed = set()
        for i in range(t + 1):
            current_claimed.update(active_nodes_history[i])
        for n in G.nodes():
            node_colors.append("red" if n in current_claimed else "gray")
        nx.draw(G, pos, with_labels=False, node_color=node_colors, node_size=50, edge_color="lightgray")
        plt.title(f"{topic_name} – Agent Influence at Step {t+1}")
        filename = os.path.join("[INSERT DIRECTORY]", f"{topic_name} – Agent(-1) Influence at Step {t+1}.png")

        plt.savefig(filename, dpi=300)
        plt.close()



In [4]:
#Computing clustering coefficient and centrality measures
clustering_coeffs = nx.clustering(G)
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)

#Converting centrality and clustering metrics to a frame for analysis
network_metrics = pd.DataFrame({
    "Node": list(G.nodes()),
    "Degree": [G.degree(n) for n in G.nodes()],
    "Clustering Coefficient": [clustering_coeffs[n] for n in G.nodes()],
    "Degree Centrality": [degree_centrality[n] for n in G.nodes()],
    "Betweenness Centrality": [betweenness_centrality[n] for n in G.nodes()]
})


#Display network metrics
print("Network Metrics (First 5 Rows):")
print(network_metrics.head())

#Saving network metrics to a CSV file 
network_metrics.to_csv("[INSERT DIRECTORY]/1Pnetwork_metricsparalleltoNEG1arbitrary.csv", index=False)


Network Metrics (First 5 Rows):
   Node  Degree  Clustering Coefficient  Degree Centrality  \
0     0       3                0.333333           0.333333   
1     1       3                0.666667           0.333333   
2     2       3                0.333333           0.333333   
3     3       4                0.666667           0.444444   
4     4       3                0.666667           0.333333   

   Betweenness Centrality  
0                0.030093  
1                0.030093  
2                0.023148  
3                0.023148  
4                0.030093  


In [5]:
#Function to run parallel simulations
def run_single_simulation(sim_idx, topic_index, push_toward, G, A_initial, l, player_config):
    player = Player(
        theta_low=player_config["theta_low"],
        theta_high=player_config["theta_high"],
        delta=player_config["delta"],
        beta=player_config["beta"],
        topic_index=topic_index,
        agent_id=1,
        push_toward=push_toward,
        l=l,
        kappa=player_config["kappa"],
        gamma=player_config["gamma"]
    )
    try:
        optimal_node, optimal_message, max_influence = player.optimize_strategy_single_agent(G)
        _, _, claimed_nodes, influence_history = simulate_diffusion(player, G, optimal_node, optimal_message)
        return {
            "Simulation": sim_idx,
            "Topic": topic_index,
            "Push Direction": push_toward,
            "Initial Node": optimal_node,
            "Optimal Message": optimal_message,
            "Total Influence": max_influence,
            "Steps": len(influence_history)
        }
    except Exception as e:
        print(f"Simulation {sim_idx} Topic {topic_index} failed: {e}")
        return None


In [12]:
#Initailizations for simulations
N_SIMULATIONS = 50
TOPICS = [0, 1, 2]
PUSH = -1  
n_jobs = -1  #available CPU cores

# Define shared config
player_config = {
    "theta_low": 0.3,
    "theta_high": 0.7,
    "delta": 0.625,
    "beta": 1,
    "kappa": 0.3,
    "gamma": 0.2
}

#Using parallel processing--> create job list
job_list = [(sim, topic, PUSH, G, A_initial, l, player_config)
            for sim in range(N_SIMULATIONS) for topic in TOPICS]

#Run in parallel
results = Parallel(n_jobs=n_jobs)(
    delayed(run_single_simulation)(*job) for job in job_list
)

#Filter for failed runs
results_clean = [r for r in results if r is not None]

#Save to DataFrame
df_results = pd.DataFrame(results_clean)
df_results.to_csv("/[INSERT DIRECTORY]/parallel_simulation_resultsArbitrarytoNEG1.csv", index=False)
print("Saved results to parallel_simulation_resultsArbitrarytoNEG1.csv")


Saved results to parallel_simulation_resultsArbitrarytoNEG1.csv
